# Preprocessing

In [2]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import OneHotEncoder

## Loading cleaned data

In [ ]:
data_df = pd.read_csv('./data/netflow_clean.csv')
feat = list(data_df.columns)
feat.remove('label')

In [12]:
data_df.head()

flow_iat_mean  flow_duration  packet_length_variance  protocol  flow_bytes  \
0            4.0            4.0                     0.0       6.0   3000000.0   
1            1.0            1.0                     0.0       6.0  12000000.0   
2            1.0            1.0                     0.0       6.0  12000000.0   
3            1.0            1.0                     0.0       6.0  12000000.0   
4            3.0            3.0                     0.0       6.0   4000000.0   

   total_backward_packets  idle_mean  psh_flag_count  subflow_fwd_bytes  \
0                     0.0        0.0             0.0               12.0   
1                     0.0        0.0             0.0               12.0   
2                     0.0        0.0             0.0               12.0   
3                     0.0        0.0             0.0               12.0   
4                     0.0        0.0             0.0               12.0   

   bwd_avg_bytes_bulk   ...    bwd_header_length  ack_flag_count  \
0                 0.0   ...                  0.0             1.0   
1                 0.0   ...                  0.0             1.0   
2                 0.0   ...                  0.0             1.0   
3                 0.0   ...                  0.0             1.0   
4                 0.0   ...                  0.0             1.0   

   flow_iat_max  subflow_fwd_packets  fwd_avg_packets_bulk  fwd_iat_max  \
0           4.0                  2.0                   0.0          4.0   
1           1.0                  2.0                   0.0          1.0   
2           1.0                  2.0                   0.0          1.0   
3           1.0                  2.0                   0.0          1.0   
4           3.0                  2.0                   0.0          3.0   

   init_win_bytes_forward  bwd_packets  min_packet_length   label  
0                   329.0          0.0                6.0  BENIGN  
1                   329.0          0.0                6.0  BENIGN  
2                   329.0          0.0                6.0  BENIGN  
3                   329.0          0.0                6.0  BENIGN  
4                   245.0          0.0                6.0  BENIGN  

[5 rows x 81 columns]

#### One hot encode categorical

In [13]:
# categ_feat = ['source_port', 'destination_port', 'protocol']

# data_df = data.itervalues().next()
# data_df = pd.get_dummies(data_df, prefix=['sp', 'dp', 'pr'], columns=categ, sparse=True)
# data_df

In [14]:
data_df.as_matrix(columns=['source_port', 'destination_port']).shape

(2657645, 2)

In [16]:
port_enc = OneHotEncoder(n_values=65536)
port_enc

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values=65536, sparse=True)

In [17]:
data_df[['source_port', 'destination_port']].as_matrix().shape

(2657645, 2)

In [19]:
 port_enc.fit_transform(data_df['source_port'].as_matrix().reshape(-1, 1)).toarray() 

MemoryError: 

In [64]:
from scipy.sparse import csc_matrix

csc = csc_matrix(np.array(
    [[0, 0, 4, 0, 0, 0],
     [1, 0, 0, 0, 2, 0],
     [2, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 0, 1],
     [4, 0, 3, 2, 0, 0]]))
print csc
csc.

  (1, 0)	1
  (2, 0)	2
  (4, 0)	4
  (0, 2)	4
  (4, 2)	3
  (2, 3)	1
  (4, 3)	2
  (1, 4)	2
  (3, 5)	1


In [ ]:
# Fit on port range 0-65535
ports = np.arange(0, 65535)
ports
port_enc.fit(ports)

#encoded_port = port_enc.transform(data_df[['source_port', 'destination_port']].values) 

#### Scaling data

In [11]:
from sklearn.preprocessing import *
import pickle

#### Dropping features having very close max and min values to avoid NaNs on scaling

In [12]:
diff = data_df[feat].max(axis=0) - data_df[feat].min(axis=0)
eps = 0.00000000001

feat = list(data_df[feat].columns[~(diff.abs() < eps)])

In [13]:
# Considering all features numerical for now  Note: source_port, destination_port, protocol are
num_feat = feat

# Using MinMaxScaler
min_max = MinMaxScaler()
# Fit scaler
min_max.fit(data_df[num_feat].as_matrix())
# Save scaler using pickle
pickle.dump(min_max, open("./data/min_max_data.p", "wb"))
# Transform data
data_df[num_feat] = pd.DataFrame(min_max.transform(
    data_df[num_feat].as_matrix()), columns=num_feat)

#### Encoding labels

In [14]:
le = LabelEncoder()
le.fit(data_df['label'].as_matrix())
pickle.dump(le, open("./data/label_encoder.p", "wb"))

data_df['label'] = pd.DataFrame(le.transform(data_df['label'].as_matrix()), columns=['label'])